In [1]:
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt


In [2]:
from cs231n.data_utils import load_CIFAR10

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=10000):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [3]:
def cnn_model_fn(features, labels, mode, config):
    # Model function for CNN model
    training = mode == tf.estimator.ModeKeys.TRAIN
    
    # Conv-Relu-Bn
    conv1 = tf.layers.conv2d(name='conv1', inputs=features['X'], filters=96, kernel_size=3, strides=1, activation=tf.nn.relu,
                            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), padding='same')
    conv1bn = tf.layers.batch_normalization(name='conv1bn', inputs=conv1, training=training)
    
     # Maxpool
    pool1 = tf.layers.max_pooling2d(name='pool1', inputs=conv1bn, pool_size=2, strides=2)
    
    # Conv-Relu-Bn
    conv2 = tf.layers.conv2d(name='conv2', inputs=pool1, filters=192, kernel_size=3, strides=1, activation=tf.nn.relu,
                            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), padding='same')
    conv2bn = tf.layers.batch_normalization(name='conv2bn', inputs=conv2, training=training)
    
    # Maxpool
    pool2 = tf.layers.max_pooling2d(name='pool2', inputs=conv2bn, pool_size=2, strides=2)                            
    
    
    # Flatten
    pool1_flatten = tf.reshape(name='pool1_flatten', tensor=pool2, shape=[-1, 8 * 8 * 192])
    
    # FC
    fc1 = tf.layers.dense(name='fc1', inputs=pool1_flatten, units=192, activation=tf.nn.relu, 
                          kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
    # Dropout
    dropout1 = tf.layers.dropout(name='dropout1',
                                  inputs=fc1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    fc2 = tf.layers.dense(name='fc2', inputs=dropout1, units=192, activation=tf.nn.relu,
                         kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
    # Dropout
    dropout2 = tf.layers.dropout(name='dropout2',
                                  inputs=fc2, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # Logits
    logits = tf.layers.dense(name='logits', inputs=dropout2, units=10, activation=None, 
                             kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
    
    predictions = {
        # Generate predictions
        'classes': tf.argmax(input=logits, axis=1),
        'probabilities': tf.nn.softmax(logits=logits, name='softmax_tensor')
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Loss
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
    
    # Training operation
    if mode == tf.estimator.ModeKeys.TRAIN:
        learning_rate = tf.train.exponential_decay(0.001, tf.train.get_global_step(),
                                           10000, 0.96, staircase=True)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        
        # batch normalization in tensorflow requires this extra dependency
        extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(extra_update_ops):
            train_op  = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Evaluation metrics
    eval_metric_ops = {
        'accuracy': tf.metrics.accuracy(labels=labels, predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [4]:
def inception_module(previous):    
    # 1x1_128 conv
    conv1_1_128 = tf.layers.conv2d(inputs=previous, filters=64, kernel_size=1, strides=1, activation=tf.nn.relu, 
                             kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), padding='same')
    
    # 1x1_64 conv
    conv1_1_64_1 = tf.layers.conv2d(inputs=previous, filters=64, kernel_size=1, strides=1, activation=tf.nn.relu, 
                             kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), padding='same')
    
    # 1x1_64 conv
    conv1_1_64_2 = tf.layers.conv2d(inputs=previous, filters=64, kernel_size=1, strides=1, activation=tf.nn.relu, 
                             kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), padding='same')
    
    # 3x3 conv
    conv3_3 = tf.layers.conv2d(inputs=conv1_1_64_1, filters=64, kernel_size=3, strides=1, activation=tf.nn.relu, 
                             kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), padding='same')
    
    # 5x5 conv
    conv5_5 = tf.layers.conv2d(inputs=conv1_1_64_2, filters=64, kernel_size=5, strides=1, activation=tf.nn.relu, 
                             kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), padding='same')
    
    # maxpooling
    pool3_3 = tf.layers.max_pooling2d(inputs=previous, pool_size=3, strides=1, padding='same')
    
    # 1x1_64 conv
    conv1_1_64_3 = tf.layers.conv2d(inputs=pool3_3, filters=64, kernel_size=1, strides=1, activation=tf.nn.relu, 
                             kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), padding='same')
    
    # 28x28x480
    return tf.concat(values=[conv1_1_128, conv3_3, conv5_5, conv1_1_64_3], axis=3)

In [5]:
def lenet_model_fn(features, labels, mode, config):
    # Model function for CNN model
    training = mode == tf.estimator.ModeKeys.TRAIN
    
    inputs = features['X']
    
    # with tf.device('/cpu:0'):
    # Conv-Relu-Bn
    conv1 = tf.layers.conv2d(inputs=inputs, filters=256, kernel_size=3, strides=1, activation=tf.nn.relu, 
                             kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), padding='valid')
    conv1bn = tf.layers.batch_normalization(inputs=conv1, training=training)

    # Conv-Relu-Bn
    conv2 = tf.layers.conv2d(inputs=conv1bn, filters=256, kernel_size=3, strides=1, activation=tf.nn.relu, 
                             kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), padding='valid')
    conv2bn = tf.layers.batch_normalization(inputs=conv2, training=training)
    
    # with tf.device('/device:GPU:0'):
    # inception module
    inc_module = inception_module(conv2bn)

    # Flatten
    pool1_flatten = tf.reshape(name='pool1_flatten', tensor=inc_module, shape=[-1, 28*28*256])

    # FC
    fc1 = tf.layers.dense(name='fc1', inputs=pool1_flatten, units=480, activation=tf.nn.relu, 
                          kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())

    # Logits
    logits = tf.layers.dense(name='logits', inputs=fc1, units=10, activation=None, 
                                 kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())

    predictions = {
        # Generate predictions
        'classes': tf.argmax(input=logits, axis=1),
        'probabilities': tf.nn.softmax(logits=logits, name='softmax_tensor')
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Loss
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
    
    # Training operation
    if mode == tf.estimator.ModeKeys.TRAIN:
        learning_rate = tf.train.exponential_decay(0.001, tf.train.get_global_step(),
                                           10000, 0.96, staircase=True)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        
        # batch normalization in tensorflow requires this extra dependency
        extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(extra_update_ops):
            train_op  = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Evaluation metrics
    eval_metric_ops = {
        'accuracy': tf.metrics.accuracy(labels=labels, predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [7]:
def vgg_model_fn(features, labels, mode, config):
    # Model function for CNN model
    training = mode == tf.estimator.ModeKeys.TRAIN
    
    inputs = features['X']
    
    # with tf.device('/cpu:0'):
    # Conv-Relu-Bn
    conv1 = tf.layers.conv2d(inputs=inputs, filters=64, kernel_size=3, strides=1, activation=tf.nn.relu, 
                             kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), padding='same')
    conv1bn = tf.layers.batch_normalization(inputs=conv1, training=training)

    # Conv-Relu-Bn
    conv2 = tf.layers.conv2d(inputs=conv1bn, filters=64, kernel_size=3, strides=1, activation=tf.nn.relu, 
                             kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), padding='same')
    conv2bn = tf.layers.batch_normalization(inputs=conv2, training=training)
                                            
    # maxpooling
    pooling1 = tf.layers.max_pooling2d(inputs=conv2bn, pool_size=2, strides=2)
    
    # Conv-Relu-Bn
    conv3 = tf.layers.conv2d(inputs=pooling1, filters=128, kernel_size=3, strides=1, activation=tf.nn.relu, 
                             kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), padding='same')
    conv3bn = tf.layers.batch_normalization(inputs=conv3, training=training)             
    
    # Conv-Relu-Bn
    conv4 = tf.layers.conv2d(inputs=conv3bn, filters=128, kernel_size=3, strides=1, activation=tf.nn.relu, 
                             kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), padding='same')
    conv4bn = tf.layers.batch_normalization(inputs=conv4, training=training)   
                                            
    # maxpooling
    pooling2 = tf.layers.max_pooling2d(inputs=conv4bn, pool_size=2, strides=2)
       
                                            
    # Conv-Relu-Bn
    conv5 = tf.layers.conv2d(inputs=pooling2, filters=256, kernel_size=3, strides=1, activation=tf.nn.relu, 
                             kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), padding='same')
    conv5bn = tf.layers.batch_normalization(inputs=conv5, training=training)             
    
    # Conv-Relu-Bn
    conv6 = tf.layers.conv2d(inputs=conv5bn, filters=256, kernel_size=3, strides=1, activation=tf.nn.relu, 
                             kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), padding='same')
    conv6bn = tf.layers.batch_normalization(inputs=conv6, training=training)   
                                            
    # maxpooling
    pooling3 = tf.layers.max_pooling2d(inputs=conv6bn, pool_size=2, strides=2)                                            

    # Flatten
    pool1_flatten = tf.reshape(tensor=pooling3, shape=[-1, 4*4*256])

    # FC
    fc1 = tf.layers.dense(name='fc1', inputs=pool1_flatten, units=1000, activation=tf.nn.relu, 
                          kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())

    # Logits
    logits = tf.layers.dense(name='logits', inputs=fc1, units=10, activation=None, 
                                 kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())

    predictions = {
        # Generate predictions
        'classes': tf.argmax(input=logits, axis=1),
        'probabilities': tf.nn.softmax(logits=logits, name='softmax_tensor')
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Loss
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
    
    # Training operation
    if mode == tf.estimator.ModeKeys.TRAIN:
        learning_rate = tf.train.exponential_decay(0.001, tf.train.get_global_step(),
                                           10000, 0.96, staircase=True)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        
        # batch normalization in tensorflow requires this extra dependency
        extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(extra_update_ops):
            train_op  = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Evaluation metrics
    eval_metric_ops = {
        'accuracy': tf.metrics.accuracy(labels=labels, predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [8]:
# configuration
session_conf = tf.ConfigProto(
    # device_count={'CPU' : 1, 'GPU' : 0},
)
session_conf.log_device_placement=True
# session_conf.gpu_options.allow_growth=True
# session_conf.gpu_options.per_process_gpu_memory_fraction=0.7

run_config = tf.estimator.RunConfig()
run_config.session_conf=session_conf

# logging
tensors_to_log = {'probabilities': 'softmax_tensor'}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)

# model function
cnn_model = tf.estimator.Estimator(model_fn=vgg_model_fn, model_dir='E:/tmp/my_model', config=run_config)

INFO:tensorflow:Using config: {'session_conf': log_device_placement: true
}


In [9]:
# train model
train_input_fun = tf.estimator.inputs.numpy_input_fn(
                            x={'X': X_train.astype(np.float32)}, 
                            y=y_train, batch_size=100, 
                            num_epochs=None, shuffle=True)
cnn_model.train(input_fn=train_input_fun, steps=20000, hooks=[])

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into E:/tmp/my_model\model.ckpt.
INFO:tensorflow:step = 1, loss = 3.76917
INFO:tensorflow:global_step/sec: 9.45671
INFO:tensorflow:step = 101, loss = 1.54222 (10.576 sec)
INFO:tensorflow:global_step/sec: 9.59374
INFO:tensorflow:step = 201, loss = 1.06099 (10.423 sec)
INFO:tensorflow:global_step/sec: 9.63239
INFO:tensorflow:step = 301, loss = 1.22074 (10.382 sec)
INFO:tensorflow:global_step/sec: 9.53919
INFO:tensorflow:step = 401, loss = 0.963321 (10.483 sec)
INFO:tensorflow:global_step/sec: 9.61672
INFO:tensorflow:step = 501, loss = 0.948941 (10.399 sec)
INFO:tensorflow:global_step/sec: 9.44375
INFO:tensorflow:step = 601, loss = 0.880274 (10.589 sec)
INFO:tensorflow:global_step/sec: 9.55071
INFO:tensorflow:step = 701, loss = 0.731768 (10.471 sec)
INFO:tensorflow:global_step/sec: 9.42741
INFO:tensorflow:step = 801, loss = 0.71446 (10.607 sec)
INFO:tensorflow:global_step/sec: 9.35376
INFO:tensorflow:step

INFO:tensorflow:step = 8101, loss = 0.076485 (10.712 sec)
INFO:tensorflow:global_step/sec: 9.33893
INFO:tensorflow:step = 8201, loss = 0.0191821 (10.726 sec)
INFO:tensorflow:global_step/sec: 9.3333
INFO:tensorflow:step = 8301, loss = 0.0251983 (10.695 sec)
INFO:tensorflow:global_step/sec: 9.36846
INFO:tensorflow:step = 8401, loss = 0.0233849 (10.675 sec)
INFO:tensorflow:global_step/sec: 9.34128
INFO:tensorflow:step = 8501, loss = 0.0242617 (10.705 sec)
INFO:tensorflow:global_step/sec: 9.34711
INFO:tensorflow:step = 8601, loss = 0.0464927 (10.699 sec)
INFO:tensorflow:global_step/sec: 9.34039
INFO:tensorflow:step = 8701, loss = 0.0315238 (10.705 sec)
INFO:tensorflow:global_step/sec: 9.33493
INFO:tensorflow:step = 8801, loss = 0.0209302 (10.712 sec)
INFO:tensorflow:global_step/sec: 9.33792
INFO:tensorflow:step = 8901, loss = 0.0644755 (10.710 sec)
INFO:tensorflow:global_step/sec: 9.33647
INFO:tensorflow:step = 9001, loss = 0.035165 (10.711 sec)
INFO:tensorflow:global_step/sec: 9.37713
INF

INFO:tensorflow:global_step/sec: 9.41115
INFO:tensorflow:step = 16201, loss = 0.0212823 (10.626 sec)
INFO:tensorflow:global_step/sec: 9.41847
INFO:tensorflow:step = 16301, loss = 0.000598357 (10.618 sec)
INFO:tensorflow:global_step/sec: 9.39335
INFO:tensorflow:step = 16401, loss = 0.135013 (10.646 sec)
INFO:tensorflow:global_step/sec: 9.33679
INFO:tensorflow:step = 16501, loss = 0.0198751 (10.741 sec)
INFO:tensorflow:global_step/sec: 9.31246
INFO:tensorflow:step = 16601, loss = 0.00124773 (10.707 sec)
INFO:tensorflow:global_step/sec: 9.35619
INFO:tensorflow:step = 16701, loss = 0.0406598 (10.688 sec)
INFO:tensorflow:Saving checkpoints for 16759 into E:/tmp/my_model\model.ckpt.
INFO:tensorflow:global_step/sec: 7.33007
INFO:tensorflow:step = 16801, loss = 0.0450893 (13.642 sec)
INFO:tensorflow:global_step/sec: 9.34411
INFO:tensorflow:step = 16901, loss = 0.0010812 (10.702 sec)
INFO:tensorflow:global_step/sec: 9.34578
INFO:tensorflow:step = 17001, loss = 0.00110149 (10.700 sec)
INFO:tenso

In [12]:
# evaluate model
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
                            x={'X': X_val.astype(np.float32)}, 
                            y=y_val, num_epochs=1, shuffle=False)
eval_results = cnn_model.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2018-01-01-17:47:57
INFO:tensorflow:Restoring parameters from E:/tmp/my_model\model.ckpt-20000
INFO:tensorflow:Finished evaluation at 2018-01-01-17:47:58
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.824, global_step = 20000, loss = 1.6192
{'global_step': 20000, 'accuracy': 0.824, 'loss': 1.6192037}
